# This NoteBook is for the GEOLOCATION to generate CSVs from RP sessions based on predefined events 

In [1]:
import os
import sys
import pprint
import csv
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import radical.utils     as ru
import radical.analytics as ra
!radical-stack


/opt/anaconda2/lib/python2.7/site-packages/radical/pilot
/opt/anaconda2/lib/python2.7/site-packages/radical/utils
/opt/anaconda2/lib/python2.7/site-packages/radical/saga
/opt/anaconda2/lib/python2.7/site-packages/radical/analytics

  python               : 2.7.14
  pythonpath           : 
  virtualenv           : 

  radical.analytics    : 0.70.0
  radical.pilot        : 0.70.0
  radical.saga         : 0.70.0
  radical.utils        : 0.70.0



# This will not work with Design2 (only Design1)

In [2]:
loc = "/home/aymen/SummerRadical/iceberg_escience/Geolocation/Data/Sessions/rp.session.js-17-21.jetstream-cloud.org.aymen.018292.0007/"
src = os.path.dirname(loc)
sid = os.path.basename(loc)
session = ra.Session(sid=sid, stype='radical.pilot', src=loc)
session.filter(etype=event_entity, inplace=True) 
dataframe=pd.DataFrame(columns=['ImageName1','ImageName2','ImageSize1','ImageSize2','TTX','UnitID'])

for unit in units.get():
        
        if unit.uid == 'unit.000000': #skipping the first unit because unit.00000 is only for image discovery
            skip = True
            continue
            
        else:
            
            exec_duration = unit.duration(event=[{ru.EVENT: 'cu_exec_start'},
                                                 {ru.EVENT: 'cu_exec_stop'}])

            durations.append(exec_duration)
            cu_dur = sum(durations) / len(durations)
            
            
            #read the Penguins_images.csv and match the 
            #image name from it with the image name from the unit.* 
            #to get the size of the image from 
            myfilepath = '/home/aymen/SummerRadical/iceberg_escience/Geolocation/Data/Desc1CSV.csv'
            with open(myfilepath) as f:
                reader = csv.reader(f) 
                next(reader) # skip header
                data = []
                for row in reader:
                    data.append(row)
                    if row[0] == unit.description["arguments"][14] :

                            try:
                                  session_data = {
                                                        'ImageName1'      : unit.description["arguments"][14],
                                                        'ImageName2'      : unit.description["arguments"][14],
                                                        'ImageSize1'      : row[1],
                                                        'ImageSize2'      : row[1],
                                                        'TTX'            : exec_duration,
                                                        'UnitID'         : unit.uid[5:]
                                                }
                            except Exception as e:
                                                print e
                                                session_data = {
                                                         'ImageName1'      : unit.description["arguments"][14],
                                                         'ImageName2'      : unit.description["arguments"][14],
                                                         'ImageSize1'      : row[1],
                                                         'ImageSize2'      : row[1],
                                                         'TTX'            : exec_duration,
                                                         'UnitID'         : unit.uid[5:]
                                                               }
                                                pass

                            dataframe =  dataframe.append(session_data, ignore_index=True)
                            dataframe =  dataframe.reset_index().drop('index', axis=1)
                            dataframe.to_csv('/home/aymen/SummerRadical/iceberg_escience/Geolocation/Data/units_durations.csv',index=False)


NameError: name 'event_entity' is not defined

In [ ]:
import os 
import sys
print